In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister


In [2]:
C = QuantumCircuit()
qubits=5

qr_people = QuantumRegister(qubits,'people')
cr_people = ClassicalRegister(qubits, 'c_people')
C.add_register(qr_people)
C.add_register(cr_people)

qr_opinion = QuantumRegister(1, 'opinion')
cr_opinion = ClassicalRegister(1, 'c_opinion')
C.add_register(qr_opinion)
C.add_register(cr_opinion)

qr_beer = QuantumRegister(1, 'beer')
cr_beer = ClassicalRegister(1, 'c_beer')
C.add_register(qr_beer)
C.add_register(cr_beer)

C.draw()

people_0: 
             
   people_1: 
             
   people_2: 
             
   people_3: 
             
   people_4: 
             
  opinion_0: 
             
     beer_0: 
             
 c_people: 5/
             
c_opinion: 1/
             
   c_beer: 1/

In [45]:
def add_oracle(circuit, qubits, number):
    ctrls = np.binary_repr(number, width=qubits)
    print(ctrls)
    for i, c in enumerate(ctrls):
        if c == '1':
            circuit.x(qubits - i - 1)
    
    circuit.mct(list(range(qubits)), qr_opinion[0])
            
    for i, c in enumerate(ctrls):
        if c == '1':
            circuit.x(qubits - i - 1)
    

In [46]:
names = ['sergio', 'carmen', 'adrian', 'alfonso']

opinion = [0, 1, 0, 1]

In [68]:
def circuit(C, qubits, opinion):
    for q in range(qubits):
        C.h(q)
    for i, o in enumerate(opinion):
        if o == 1:
            add_oracle(C, qubits, i)
    C.barrier()
            
def circuit_inverse(C, qubits, opinion):
    opinion.reverse()
    for i, o in enumerate(opinion):
        if o == 1:
            add_oracle(C, qubits, 2**qubits - 1 - i)
    for q in range(qubits):
        C.h(q)
    opinion.reverse()
    C.barrier()
            
def oracle(C):
    C.z(qr_opinion)
    C.barrier()
    
def diffusion(C, qubits):
    C.h(qr_opinion)
    C.mct(list(range(qubits)), qr_opinion[0])
    C.h(qr_opinion)
    C.barrier()

In [69]:
def create_circuit(qubits, iterations, opinion, beers=True):
    C = QuantumCircuit()

    qr_people = QuantumRegister(qubits,'people')
    cr_people = ClassicalRegister(qubits, 'c_people')
    C.add_register(qr_people)
    C.add_register(cr_people)

    qr_opinion = QuantumRegister(1, 'opinion')
    cr_opinion = ClassicalRegister(1, 'c_opinion')
    C.add_register(qr_opinion)
    C.add_register(cr_opinion)

    qr_beer = QuantumRegister(1, 'beer')
    cr_beer = ClassicalRegister(1, 'c_beer')
    C.add_register(qr_beer)
    C.add_register(cr_beer)
    
    
    if beers:
        C.h(qr_beer)
        C.barrier()
        
    circuit(C, qubits, opinion)
    
    for i in range(iterations):
        oracle(C)
        circuit_inverse(C, qubits, opinion)
        diffusion(C, qubits)
        circuit(C, qubits, opinion)
        
    if beers:
        C.barrier()
        C.h(qr_beer)
        
    for i in range(qubits + 2):
        C.measure(i,i)
        
    return C

In [70]:
C = create_circuit(2, 2, [0,1,1,0])

C.draw()

01
10
10
01
01
10
10
01
01
10


░ ┌───┐┌───┐     ┌───┐           ░       ░           ┌───┐»
   people_0: ──────░─┤ H ├┤ X ├──■──┤ X ├──■────────░───────░────────■──┤ X ├»
                   ░ ├───┤└───┘  │  ├───┤  │  ┌───┐ ░       ░ ┌───┐  │  ├───┤»
   people_1: ──────░─┤ H ├───────■──┤ X ├──■──┤ X ├─░───────░─┤ X ├──■──┤ X ├»
                   ░ └───┘     ┌─┴─┐└───┘┌─┴─┐└───┘ ░ ┌───┐ ░ └───┘┌─┴─┐└───┘»
  opinion_0: ──────░───────────┤ X ├─────┤ X ├──────░─┤ Z ├─░──────┤ X ├─────»
             ┌───┐ ░           └───┘     └───┘      ░ └───┘ ░      └───┘     »
     beer_0: ┤ H ├─░────────────────────────────────░───────░────────────────»
             └───┘ ░                                ░       ░                »
 c_people: 2/════════════════════════════════════════════════════════════════»
                                                                             »
c_opinion: 1/════════════════════════════════════════════════════════════════»
                                                                             »
   c_beer: 1/════════════════════════════════════════════════════════════════»
                                                                             »
«                  ┌───┐┌───┐ ░                 ░ ┌───┐┌───┐     ┌───┐     »
«   people_0: ──■──┤ X ├┤ H ├─░────────■────────░─┤ H ├┤ X ├──■──┤ X ├──■──»
«               │  ├───┤└───┘ ░        │        ░ ├───┤└───┘  │  ├───┤  │  »
«   people_1: ──■──┤ H ├──────░────────■────────░─┤ H ├───────■──┤ X ├──■──»
«             ┌─┴─┐└───┘      ░ ┌───┐┌─┴─┐┌───┐ ░ └───┘     ┌─┴─┐└───┘┌─┴─┐»
«  opinion_0: ┤ X ├───────────░─┤ H ├┤ X ├┤ H ├─░───────────┤ X ├─────┤ X ├»
«             └───┘           ░ └───┘└───┘└───┘ ░           └───┘     └───┘»
«     beer_0: ────────────────░─────────────────░──────────────────────────»
«                             ░                 ░                          »
« c_people: 2/═════════════════════════════════════════════════════════════»
«                                                                          »
«c_opinion: 1/═════════════════════════════════════════════════════════════»
«                                                                          »
«   c_beer: 1/═════════════════════════════════════════════════════════════»
«                                                                          »
«                   ░       ░           ┌───┐     ┌───┐┌───┐ ░                »
«   people_0: ──────░───────░────────■──┤ X ├──■──┤ X ├┤ H ├─░────────■───────»
«             ┌───┐ ░       ░ ┌───┐  │  ├───┤  │  ├───┤└───┘ ░        │       »
«   people_1: ┤ X ├─░───────░─┤ X ├──■──┤ X ├──■──┤ H ├──────░────────■───────»
«             └───┘ ░ ┌───┐ ░ └───┘┌─┴─┐└───┘┌─┴─┐└───┘      ░ ┌───┐┌─┴─┐┌───┐»
«  opinion_0: ──────░─┤ Z ├─░──────┤ X ├─────┤ X ├───────────░─┤ H ├┤ X ├┤ H ├»
«                   ░ └───┘ ░      └───┘     └───┘           ░ └───┘└───┘└───┘»
«     beer_0: ──────░───────░────────────────────────────────░────────────────»
«                   ░       ░                                ░                »
« c_people: 2/════════════════════════════════════════════════════════════════»
«                                                                             »
«c_opinion: 1/════════════════════════════════════════════════════════════════»
«                                                                             »
«   c_beer: 1/════════════════════════════════════════════════════════════════»
«                                                                             »
«              ░ ┌───┐┌───┐     ┌───┐           ░  ░      ┌─┐         
«   people_0: ─░─┤ H ├┤ X ├──■──┤ X ├──■────────░──░──────┤M├─────────
«              ░ ├───┤└───┘  │  ├───┤  │  ┌───┐ ░  ░      └╥┘┌─┐      
«   people_1: ─░─┤ H ├───────■──┤ X ├──■──┤ X ├─░──░───────╫─┤M├──────
«              ░ └───┘     ┌─┴─┐└───┘┌─┴─┐└───┘ ░  ░       ║ └╥┘┌─┐   
«  opinion_0: ─░───────────┤ X ├─────┤ X ├──────░──░───────╫──╫─┤M├───
«              ░           └───┘     └───┘      ░  ░ 

In [61]:
type(opinion.reverse())

NoneType